# Final Project: Returning to Twitter Data
## Twitter: An Analysis in Linguistic Diversity

In the past century, human mobility is at its greatest since our dawn. This mobility begets diverse hotspots across the landscape. However, some places are more diverse than others. A copule of questions should come to mind. What is meant when we call a place diverse? Why type of diversity? In this case, we are talking about the topography of langauges for particular cities across the United States, but we could be measuring ethnic or socioeconomic diversity. How do we measure diversity? These are just a few of the questions that we will be exploring as we begin to formalize our understanding of big data manipulation.

----

### Linguistic Diversity and Twitter's role

What is linguistic diversity? For our purposes, linguistic diversity is a some sort of measure that accounts for both the number of languages spoken in a particular area as well as the number of speakers per language. This is where Twitter comes in. Collecting language data on a region, in this case cities, is particularly difficult, and if you ware wanting to find an open source dataset with such linguistic information, you would be hard pressed to find one that counts up past five distinct languages. Twitter, however, provides the language spoken in the Tweet as an attribute accessible through their API. It also allows a user to collect on tweets within a specified region by specifying a geographic radius around a city. This gives us the unique ability to analyze the linguistic landscape of a given region. This, of course, can be compared between locations or evaluated across time splits in the data. 

It is also important to note that this may not be representative of any given physical location. In fact, it almost certainly overweights English's speakers compared to others. A large proportion of Twitter is written in English, more so than there are native English speakers. Yet, this doesn't mean that there isn't any connection to the physical landscape. In fact, all things being equal, the expectation would be that more diverse Twitter cities are also more diverse physical cites and vice versa. However, we won't be making such assumptions throughout these notebooks. To do so would take some sort of validation which is outside of the purview of these lessons.

----

### The Data

The Twitter data are stored in a Postgres database and contains several tables. The primary table that we will be working with is the `tweet` table, however, there are also a `hashtag`, `mention`, `url` and `job` table. We will come back to these, but for the time being, let's go over the `tweet` table and its attributes.


attribute     | description
--------------|------------
`tweet_id_str`| tweet's identifier
`job_id`      | job identifier (pertaining to geographic location)
`created_at`  | when the tweet was written
`text`        | the text of the tweet
`from_user`   | user id who created the tweet
`from_user_name`| username of tweet creator
`from_user_created_at`| the date the user was created
`from_user_followers`| number of followers the user has
`from_user_following`| number of people the user is following
`from_user_favorites`| sum of likes of user's tweets
`to_user`     | the id of the person the user tweeted at
`to_user_name`| the user name of the person tweeted at
`location_geo`| the coordinates of where the tweet was sent out
`iso_language`| the language of the tweet

We can also query for the column names of the table and their data type...

# Task 1)

Using Spark and Drill, access a set of 10,000 tweets each from 3 different regions.

In [2]:

import time
from pydrill.client import PyDrill
import psycopg2
import pandas as pd


job_id: 273 = Chicago, 291 = New York, 303 = Seattle

In [2]:

drill = PyDrill(host='c04.h-spark.cgi.missouri.edu', port=8047)

if not drill.is_active():
    raise ImproperlyConfigured('Please run Drill first')
    
statement = '''
(SELECT tweet_id_str, job_id, created_at, from_user, iso_language FROM dfs.datasets.`twitter/tweet.json`
WHERE job_id = 273 LIMIT 10000)

UNION

(SELECT tweet_id_str, job_id, created_at, from_user, iso_language FROM dfs.datasets.`twitter/tweet.json`
WHERE job_id = 291 LIMIT 10000)

UNION

(SELECT tweet_id_str, job_id, created_at, from_user, iso_language FROM dfs.datasets.`twitter/tweet.json`
WHERE job_id = 303 LIMIT 10000)
'''

rows = drill.query(statement, timeout = 60)

task1 = rows.to_dataframe()
task1.head(10)


,created_at,from_user,iso_language,job_id,tweet_id_str
0,2017-02-15T13:42:37,1804911194,en,273,831861275193925636
1,2017-02-15T13:42:38,177371816,en,273,831861275806330880
2,2017-02-15T13:42:38,1564169448,en,273,831861275873386497
3,2017-02-15T13:42:38,2218167267,en,273,831861276951400450
4,2017-02-15T13:42:38,708075117746769920,en,273,831861277232336896
5,2017-02-15T13:42:38,2320386275,en,273,831861277614018560
6,2017-02-15T13:42:38,387991678,en,273,831861279014858754
7,2017-02-15T13:42:38,1633912681,en,273,831861279040139265
8,2017-02-15T13:42:39,795384709488349184,en,273,831861281342779394
9,2017-02-15T13:42:39,1952351262,en,273,831861281397301248


### <span style="background:yellow">For each of the remaining tasks, add execution cells as you see fit.  Please keep them grouped with the task.</span>

# Task 2)
Produce basic analysis to identify how the use of language is different in these three regions. 

Using Spark and Drill, we will query 10,000 tweets from each region, then limit the set to one tweet per individual tweeter for each language (one tweet per language for multilingual tweeters). We will also remove all unidentifiable language tweets. Finally we will count the number of tweets for each language in each region, and compare.

In [3]:

drill = PyDrill(host='c04.h-spark.cgi.missouri.edu', port=8047)

if not drill.is_active():
    raise ImproperlyConfigured('Please run Drill first')

statement = '''
SELECT DISTINCT iso_language as `language`, job_id as `city`, COUNT(*) as `count`
FROM(

SELECT DISTINCT from_user, iso_language, job_id
FROM(

(SELECT job_id, from_user, iso_language FROM dfs.datasets.`twitter/tweet.json`
WHERE job_id = 273 LIMIT 10000)
UNION
(SELECT job_id, from_user, iso_language FROM dfs.datasets.`twitter/tweet.json`
WHERE job_id = 291 LIMIT 10000)
UNION
(SELECT job_id, from_user, iso_language FROM dfs.datasets.`twitter/tweet.json`
WHERE job_id = 303 LIMIT 10000)
)AS a

WHERE iso_language <> 'und'
)AS b

GROUP BY iso_language, job_id
ORDER BY iso_language, job_id DESC
'''

rows = drill.query(statement, timeout = 100)

task2a = rows.to_dataframe()
task2a

,city,count,language
0,303,27,ar
1,291,35,ar
2,273,16,ar
3,303,3,cs
4,291,1,cs
5,273,2,cs
6,303,4,cy
7,291,1,cy
8,273,3,cy
9,303,5,da


We will repeat the last query, this time using Postgres.

In [4]:

statement = """
SELECT DISTINCT iso_language as language, job_id as city, COUNT(*) as count
FROM(

SELECT DISTINCT from_user, iso_language, job_id
FROM(

(SELECT job_id, from_user, iso_language FROM twitter.tweet
WHERE job_id = 273 LIMIT 10000)
UNION
(SELECT job_id, from_user, iso_language FROM twitter.tweet
WHERE job_id = 291 LIMIT 10000)
UNION
(SELECT job_id, from_user, iso_language FROM twitter.tweet
WHERE job_id = 303 LIMIT 10000)
)AS a

WHERE iso_language <> 'und'
)AS b

GROUP BY iso_language, job_id
ORDER BY iso_language, job_id DESC;
"""

try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=60s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task2b = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)


Here, we will import the Shannon Diversity Index function and run it on our Postgres results to measure the linguistic
diversity of each region.

In [5]:

from skbio.diversity.alpha import shannon


In [6]:

task2b['count'].groupby(task2b['city']).apply(shannon)


city
273    0.412501
291    0.452917
303    0.547235
Name: count, dtype: float64

We see that Seattle is the most diverse, while Chicago is the least diverse. This seems to be plausible since New York and Seattle are both coastal cities with large immigrant populations. Although it is a bit surprising that Seattle is more diverse than New York, as New York is often thought of as the immigration capital of the U.S.

# Task 3)

Produce an analysis that contrasts the diversity of hashtags in each of 3 regions with the diversity of language. Use Postgres. 

Continuing with Postgres, we will count the number of tweets with hashtags per each identifiable language, for each region. We will do this by joining the tweet and hashtag tables. Our results will continue to be limited to one tweet per tweeter, per language.

a. Chicago

In [7]:

statement = """
SELECT DISTINCT iso_language as language, COUNT(*) as hashtag_count
FROM(

SELECT DISTINCT from_user, iso_language
FROM(

(SELECT t.from_user, t.iso_language FROM twitter.tweet t
INNER JOIN twitter.hashtag h
ON t.tweet_id_str = h.tweet_id
WHERE t.job_id = 273 LIMIT 10000)

)AS a

WHERE iso_language <> 'und'
)AS b

GROUP BY iso_language
ORDER BY COUNT(*) DESC
"""

try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=60s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task3a = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task3a


,language,hashtag_count
0,en,4284
1,tl,24
2,es,21
3,fr,13
4,ht,8
5,lt,8
6,de,7
7,et,6
8,in,4
9,it,4


b. New York

In [8]:

statement = """
SELECT DISTINCT iso_language as language, COUNT(*) as hashtag_count
FROM(

SELECT DISTINCT from_user, iso_language
FROM(

(SELECT t.from_user, t.iso_language FROM twitter.tweet t
INNER JOIN twitter.hashtag h
ON t.tweet_id_str = h.tweet_id
WHERE t.job_id = 291 LIMIT 10000)

)AS a

WHERE iso_language <> 'und'
)AS b

GROUP BY iso_language
ORDER BY COUNT(*) DESC
"""

try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=60s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task3b = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task3b


,language,hashtag_count
0,en,4111
1,es,82
2,fr,22
3,ja,22
4,tl,18
5,de,16
6,ar,14
7,pt,13
8,pl,10
9,da,9


c. Seattle

In [9]:

statement = """
SELECT DISTINCT iso_language as language, COUNT(*) as hashtag_count
FROM(

SELECT DISTINCT from_user, iso_language
FROM(

(SELECT t.from_user, t.iso_language FROM twitter.tweet t
INNER JOIN twitter.hashtag h
ON t.tweet_id_str = h.tweet_id
WHERE t.job_id = 303 LIMIT 10000)

)AS a

WHERE iso_language <> 'und'
)AS b

GROUP BY iso_language
ORDER BY COUNT(*) DESC
"""

try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=60s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task3c = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task3c


,language,hashtag_count
0,en,3143
1,es,21
2,tl,18
3,fr,16
4,th,13
5,in,12
6,pt,11
7,de,8
8,it,8
9,ko,7


It's not surprising that tweets in English have the most hashtags, English tweets outweigh all other languages by a large margin. It's also not shocking to see "es" (Spanish) or "fr" (French) ranked highly in all three regions. What is interesting (to this observer at least) is that "tl" (Tagalog), the national language of the Philippines, came in second in Chicago, third in Seattle, and fourth in New York. 

# Task 4)

For each of the three regions, how are most popular hashtags different in February, 2017, compared to April, 2017?


This time we will query 100,000 tweets with hashtags from each region by joining the tweet table and hashtag table, but will only select tweets from February 2017 or April 2017 for each query. This will be accomplished using the OVERLAPS function on the "created_at" variable from the tweet table. We will count the number of tweets for each hashtag and determine the most popular 10 for each month.

a. Chicago, February 2017

In [10]:

statement= """
SELECT DISTINCT text as hashtag, COUNT(*) as count
FROM(

SELECT t.job_id, h.text
FROM twitter.tweet t
JOIN twitter.hashtag h
ON t.tweet_id_str = h.tweet_id
WHERE t.job_id = 271 AND (t.created_at, t.created_at) OVERLAPS (DATE '2017-02-01', DATE '2017-03-01')
LIMIT 100000

)AS a
GROUP BY text
ORDER BY count DESC;

"""


try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=1200s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task4a = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task4a.head(10)


,hashtag,count
0,Uber,2112
1,LasVegas,1273
2,Lyft,1240
3,MAGA,1234
4,RETWEET,1086
5,tcot,1065
6,uber,977
7,ubercode,867
8,UberPromo,843
9,Vegas,814


b. Chicago, April 2017

In [11]:

statement= """
SELECT DISTINCT text as hashtag, COUNT(*) as count
FROM(

SELECT t.job_id, h.text
FROM twitter.tweet t
JOIN twitter.hashtag h
ON t.tweet_id_str = h.tweet_id
WHERE t.job_id = 271 AND (t.created_at, t.created_at) OVERLAPS (DATE '2017-04-01', DATE '2017-05-01')
LIMIT 100000

)AS a
GROUP BY text
ORDER BY count DESC;

"""


try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=1200s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task4b = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task4b.head(10)


,hashtag,count
0,quotes,2019
1,MAGA,1938
2,tcot,1528
3,RETWEET,1525
4,job,1183
5,AprilFoolsDay,1114
6,BuildThatWall,1034
7,Review,1015
8,hiring,974
9,LasVegas,849


c. New York, February 2017

In [12]:

statement= """
SELECT DISTINCT text as hashtag, COUNT(*) as count
FROM(

SELECT t.job_id, h.text
FROM twitter.tweet t
JOIN twitter.hashtag h
ON t.tweet_id_str = h.tweet_id
WHERE t.job_id = 291 AND (t.created_at, t.created_at) OVERLAPS (DATE '2017-02-01', DATE '2017-03-01')
LIMIT 100000

)AS a
GROUP BY text
ORDER BY count DESC;

"""


try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=1200s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task4c = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task4c.head(10)

,hashtag,count
0,ShePersisted,966
1,MAGA,810
2,InfinityWar,710
3,Trump,696
4,Obamacare,625
5,OneDBestFans,608
6,iHeartAwards,601
7,Dobbs,600
8,BlackHistoryMonth,516
9,NYC,514


d. New York, April 2017

In [13]:

statement= """
SELECT DISTINCT text as hashtag, COUNT(*) as count
FROM(

SELECT t.job_id, h.text
FROM twitter.tweet t
JOIN twitter.hashtag h
ON t.tweet_id_str = h.tweet_id
WHERE t.job_id = 291 AND (t.created_at, t.created_at) OVERLAPS (DATE '2017-04-01', DATE '2017-05-01')
LIMIT 100000

)AS a
GROUP BY text
ORDER BY count DESC;

"""


try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=1200s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task4d = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task4d.head(10)

,hashtag,count
0,AMJoy,1579
1,Wrestlemania,1541
2,MAGA,1150
3,Trump,734
4,EqualPayDay,725
5,WrestleMania,710
6,Dobbs,555
7,Wings,397
8,SusanRice,378
9,TrumpTrain,363


e. Seattle, February 2017

In [14]:

statement= """
SELECT DISTINCT text as hashtag, COUNT(*) as count
FROM(

SELECT t.job_id, h.text
FROM twitter.tweet t
JOIN twitter.hashtag h
ON t.tweet_id_str = h.tweet_id
WHERE t.job_id = 303 AND (t.created_at, t.created_at) OVERLAPS (DATE '2017-02-01', DATE '2017-03-01')
LIMIT 100000

)AS a
GROUP BY text
ORDER BY count DESC;

"""


try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=1200s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task4e = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task4e.head(10)

,hashtag,count
0,Seattle,2151
1,LizaSoberano,981
2,KCAPinoyStar,902
3,MyExAndWhys6DaysToGo,806
4,giveaway,701
5,Seahawks,632
6,job,621
7,NowPlaying,584
8,NoDAPL,577
9,Alz,564


f. Seattle, April 2017

In [15]:

statement= """
SELECT DISTINCT text as hashtag, COUNT(*) as count
FROM(

SELECT t.job_id, h.text
FROM twitter.tweet t
JOIN twitter.hashtag h
ON t.tweet_id_str = h.tweet_id
WHERE t.job_id = 303 AND (t.created_at, t.created_at) OVERLAPS (DATE '2017-04-01', DATE '2017-05-01')
LIMIT 100000

)AS a
GROUP BY text
ORDER BY count DESC;

"""


try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=1200s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task4f = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task4f.head(10)

,hashtag,count
0,Seattle,1808
1,SEAvATL,909
2,ashpc2017,809
3,job,793
4,FinalFour,786
5,Alz,764
6,NCT,721
7,도영,690
8,DOYOUNG,690
9,NCT127,679


# Task 5)

There is a table in the Postgres database called "mentions". This table contains information about Twitter accounts most commonly mentioned in tweets. Identify the 20 most mentioned accounts in tweets from each of your 3 regions. 

We will accomplish this by querying 10,000 tweets that mention the screen name of another account, group the mentioned screen names and count the number of times mentioned for each.

a. Chicago

In [16]:

statement= """

SELECT DISTINCT screen_name, COUNT(*) as count
FROM twitter.mention
WHERE job_id = 273
GROUP BY screen_name
ORDER BY COUNT(*) DESC
LIMIT 10000;
"""

try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=600s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task5a = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task5a.head(20)


,screen_name,count
0,MatthewACherry,561680
1,charliekirk11,310753
2,hyped_resonance,300089
3,Cubs,291627
4,iamwilliewill,226196
5,YouTube,159265
6,chicagobulls,145749
7,BlackMarvelGirl,144128
8,KingTrillaX,134417
9,realDonaldTrump,127086


b. New York

In [17]:

statement= """

SELECT DISTINCT screen_name, COUNT(*) as count
FROM twitter.mention
WHERE job_id = 291
GROUP BY screen_name
ORDER BY COUNT(*) DESC
LIMIT 10000;
"""

try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=600s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task5b = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task5b.head(20)

,screen_name,count
0,kylegriffin1,113064
1,nytimes,105019
2,BraddJaffy,90991
3,JoyAnnReid,89020
4,realDonaldTrump,71838
5,chrislhayes,58877
6,dodo,56139
7,AnnCoulter,50674
8,Amy_Siskind,46870
9,XXL,44615


c. Seattle

In [18]:

statement= """

SELECT DISTINCT screen_name, COUNT(*) as count
FROM twitter.mention
WHERE job_id = 303
GROUP BY screen_name
ORDER BY COUNT(*) DESC
LIMIT 10000;
"""

try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=600s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task5c = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task5c.head(20)

,screen_name,count
0,shelbytriglia,109702
1,BillGates,97904
2,YouTube,84799
3,realDonaldTrump,77873
4,Onision,59408
5,frappuccino,59212
6,JCrossover,58349
7,Hisroyaldopenez,52068
8,LeftAtLondon,49169
9,IMDb,48328


We see that Chicago sports team accounts are mentioned often in that region, as is the Seahawks's account around Seattle. New York sports teams are noticably missing from the top 20 most frequent mentions around New York, but probably because New Yorkers are too busy talking about politics. Numerous political figures and pundits are mentioned in all regions, but are predominant in the Big Apple's top 20. Bill Gates is popular in Seattle, which makes sense given Microsoft's headquarters is in that region. And finally, YouTube's account showed up on the list for all three regions.

# Task 6)

Repeat task 5 using the tweet file on Spark using Drill. 

We will repeat the same exercise as task 5, but this time on Spark using Drill.

a. Chicago

In [19]:

drill = PyDrill(host='c04.h-spark.cgi.missouri.edu', port=8047)

if not drill.is_active():
    raise ImproperlyConfigured('Please run Drill first')
    
statement = '''
SELECT DISTINCT screen_name, COUNT(*) as `count`
FROM dfs.datasets.`twitter/mention.json`
WHERE job_id = 273 
GROUP BY screen_name
ORDER BY COUNT(*) DESC
LIMIT 10000

'''

rows = drill.query(statement, timeout = 600)

task6a = rows.to_dataframe()
task6a.head(20)




,count,screen_name
0,561680,MatthewACherry
1,310753,charliekirk11
2,300089,hyped_resonance
3,291627,Cubs
4,226196,iamwilliewill
5,159265,YouTube
6,145749,chicagobulls
7,144128,BlackMarvelGirl
8,134417,KingTrillaX
9,127086,realDonaldTrump


b. New York

In [20]:

drill = PyDrill(host='c04.h-spark.cgi.missouri.edu', port=8047)

if not drill.is_active():
    raise ImproperlyConfigured('Please run Drill first')
    
statement = '''
SELECT DISTINCT screen_name, COUNT(*) as `count`
FROM dfs.datasets.`twitter/mention.json`
WHERE job_id = 291 
GROUP BY screen_name
ORDER BY COUNT(*) DESC
LIMIT 10000

'''

rows = drill.query(statement, timeout = 600)

task6b = rows.to_dataframe()
task6b.head(20)

,count,screen_name
0,113064,kylegriffin1
1,105019,nytimes
2,90991,BraddJaffy
3,89020,JoyAnnReid
4,71838,realDonaldTrump
5,58877,chrislhayes
6,56139,dodo
7,50674,AnnCoulter
8,46870,Amy_Siskind
9,44615,XXL


c. Seattle

In [21]:

drill = PyDrill(host='c04.h-spark.cgi.missouri.edu', port=8047)

if not drill.is_active():
    raise ImproperlyConfigured('Please run Drill first')
    
statement = '''
SELECT DISTINCT screen_name, COUNT(*) as `count`
FROM dfs.datasets.`twitter/mention.json`
WHERE job_id = 303 
GROUP BY screen_name
ORDER BY COUNT(*) DESC
LIMIT 10000

'''

rows = drill.query(statement, timeout = 600)

task6c = rows.to_dataframe()
task6c.head(20)

,count,screen_name
0,109702,shelbytriglia
1,97904,BillGates
2,84799,YouTube
3,77873,realDonaldTrump
4,59408,Onision
5,59212,frappuccino
6,58349,JCrossover
7,52068,Hisroyaldopenez
8,49169,LeftAtLondon
9,48328,IMDb


The results here are different than those from task 5, but not vastly so. This is to be expected when querying a random set.

# Task 7)

Identify the most retweeted users in each of your regions.

Behind the scenes of a retweet, the text actually starts with "RT", followed by a space, then the retweeted account's screen name, followed by a colon. To accomplish the requested task we will select the text from a queried 10,000 tweets for each region that start with "RT", then cut the text out from the beginning of the retweet to the colon. This will give us the retweeted account. We'll then group by the screen names, and count the retweets for each.

a. Chicago

In [22]:


statement= """ 
SELECT SPLIT_PART(text,':',1) as split, COUNT(*) as count
FROM(

SELECT text
FROM twitter.tweet 
WHERE text LIKE 'RT%' AND job_id = 273
LIMIT 10000

)AS a

GROUP BY split
ORDER BY count DESC
"""


try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=600s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task7a = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task7a.head(10)


,split,count
0,RT @MichaelDavSmith,659
1,RT @MatthewACherry,586
2,RT @BrandonDaGawd,351
3,RT @hyped_resonance,168
4,RT @B_Meson,152
5,RT @COOLEEBRAVO,144
6,RT @fakeshoredrive,131
7,RT @ViralSeason,121
8,RT @Cubs,104
9,RT @dreamhustlecode,92


b. New York

In [23]:

statement= """ 
SELECT SPLIT_PART(text,':',1) as split, COUNT(*) as count
FROM(

SELECT text
FROM twitter.tweet 
WHERE text LIKE 'RT%' AND job_id = 291
LIMIT 10000

)AS a

GROUP BY split
ORDER BY count DESC
"""


try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=600s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task7b = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task7b.head(10)


,split,count
0,RT @JoyAnnReid,418
1,RT @BraddJaffy,254
2,RT @AnnCoulter,231
3,RT @kylegriffin1,179
4,RT @justinhendrix,157
5,RT @mollycrabapple,156
6,RT @nytimes,139
7,RT @LanaDelRey,119
8,RT @KillPots,119
9,RT @ltsChuckBass,114


c. Seattle

In [24]:

statement= """ 
SELECT SPLIT_PART(text,':',1) as split, COUNT(*) as count
FROM(

SELECT text
FROM twitter.tweet 
WHERE text LIKE 'RT%' AND job_id = 303
LIMIT 10000

)AS a

GROUP BY split
ORDER BY count DESC
"""


try:
    connect_str = """dbname='twitter' user='dsa_ro_user' 
    host='dbase.dsa.missouri.edu'password='readonly' options='-c statement_timeout=600s'"""
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    
    # execute the the query statement using the connection string and put it directly into a pandas dataframe
    task7c = pd.read_sql_query(statement, conn)
    
    # remember to close the connection
    conn.close()
    
except Exception as e:
    print("Uh oh, we have a problem. If it is a connection error," + 
          "is it an invalid dbname, user or password? Is it a timeout? A bad query?")
    print(e)

task7c.head(10)

,split,count
0,RT @HobbesVoltaire,402
1,RT @LadyOnTheMuna2,399
2,RT @jimmyhawk9,278
3,RT @coachporter8,178
4,RT @EarlBlackman,150
5,RT @shelbytriglia,142
6,RT @KenJennings,140
7,RT @BillGates,140
8,RT @fakedansavage,120
9,RT @LeftAtLondon,119


We see several familiar screen names from our mentioned tasks: @BillGates, @kylegriffin1, @Cubs, but don't actually see any overlap between the regions. It would appear the most popular retweeted accounts are region specific, according to this analysis.

# SAVE YOUR NOTEBOOK